# Graph Database - Hands On
In the following sections you will get a deep dive in Cypher Query Language and how to add, update and remove data of your graph database.

The example consists of:
- 3 students
- 1 professor
- 3 cities
- 1 university

# Check for Imports

In [25]:
import importlib.util

packages = ["py2neo", "pandas"]

for package in packages:
    if importlib.util.find_spec(package) is None:
        print(f"{package} is not installed. Installing now...")
        !pip install {package}
    else:
        print(f"{package} is already installed.")



py2neo is already installed.
pandas is already installed.


### Init connection to the database

In [17]:
from py2neo import Graph

DATABASE_URL = "bolt://localhost:7687"
USERNAME = "neo4j"
PASSWORD = "test1234"

graph = Graph(DATABASE_URL, auth=(USERNAME, PASSWORD))

if graph:
    print("Connected to the database")

Connected to the database


### Optionally: Reset Graph Database

In [18]:
reset_graph_query = "MATCH (n) DETACH DELETE n"

graph.run(reset_graph_query)

print("Database resetted successfully")

Database resetted successfully


### Create nodes

In [19]:
# Define queries to create person nodes by using Cypher
student_node_1 = "CREATE (p:Person:Student {name: 'John Doe', age: 20}) RETURN p"
student_node_2 = "CREATE (p:Person:Student {name: 'Leon Marshall', age: 22}) RETURN p"
student_node_3 = "CREATE (p:Person:Student {name: 'Oliver Watts', age: 19}) RETURN p"
professor_node = "CREATE (p:Person:Professor {name: 'Aaron Moss', age: 55}) RETURN p"
city_node_1 = "CREATE (c:City {name: 'Stuttgart'}) RETURN c"
city_node_2 = "CREATE (c:City {name: 'Karlsruhe'}) RETURN c"
city_node_3 = "CREATE (c:City {name: 'Reutlingen'}) RETURN c"
university_node = "CREATE (u:University {name: 'DHBW Stuttgart'}) RETURN u"

# Create person nodes in the database
graph.run(student_node_1)
graph.run(student_node_2)
graph.run(student_node_3)
graph.run(professor_node)
graph.run(city_node_1)
graph.run(city_node_2)
graph.run(city_node_3)
graph.run(university_node)

print("Nodes created successfully")

Nodes created successfully


### Get all Person Nodes

In [20]:
# Define query to get all person nodes
get_all_person_nodes = "MATCH (p:Person) RETURN labels(p) as Label, p.name AS name, p.age AS age"

# Get all person nodes from the database
result = graph.run(get_all_person_nodes).to_data_frame()

# Print all person nodes
result

,Label,name,age
0,"[Person, Student]",John Doe,20
1,"[Person, Student]",Leon Marshall,22
2,"[Person, Student]",Oliver Watts,19
3,"[Person, Professor]",Aaron Moss,55


### Get all City Nodes 

In [21]:
# Define query to get all city nodes
get_all_person_nodes = "MATCH (c:City) RETURN c.name AS name"

# Get all city nodes from the database
result = graph.run(get_all_person_nodes).to_data_frame()

# Print all city nodes
result

,name
0,Stuttgart
1,Karlsruhe
2,Reutlingen


### Get all University Nodes

In [22]:
# Define query to get all university nodes
get_all_university_nodes = "MATCH (u:University) RETURN u.name AS name"

# Get all university nodes from the database
result = graph.run(get_all_university_nodes).to_data_frame()

# Print all university nodes
result

,name
0,DHBW Stuttgart


### Create relationships

In [23]:
# Define query to create relationship between university and city node
university_city_rel = """
MATCH (u:University {name: 'DHBW Stuttgart'}), (c:City {name: 'Stuttgart'})
CREATE (u)-[:LOCATED_IN]->(c)
"""

# Relationship between professor and students
professor_students_rel = """
MATCH (prof:Professor {name: 'Aaron Moss'}), (student1:Student {name: 'John Doe'}), (student2:Student {name: 'Leon Marshall'}), (student3:Student {name: 'Oliver Watts'})
CREATE (prof)-[:TEACHES]->(student1), (prof)-[:TEACHES]->(student2), (prof)-[:TEACHES]->(student3)
"""

# Relationship between professor and city
professor_city_rel = """
MATCH (prof:Professor {name: 'Aaron Moss'}), (c:City {name: 'Stuttgart'})
CREATE (prof)-[:LIVES_IN]->(c)
"""

# Relationship between professor and university
professor_university_rel = """
MATCH (prof:Professor {name: 'Aaron Moss'}), (u:University {name: 'DHBW Stuttgart'})
CREATE (prof)-[:WORKS_AT]->(u)
"""

# Relationship between student and city
student_city_rel_1 = """
MATCH (student:Student {name: 'John Doe'}), (c:City {name: 'Karlsruhe'})
CREATE (student)-[:LIVES_IN]->(c)
"""
student_city_rel_2 = """ 
MATCH (student:Student {name: 'Leon Marshall'}), (c:City {name: 'Stuttgart'})
CREATE (student)-[:LIVES_IN]->(c)
"""
student_city_rel_3 = """
MATCH (student:Student {name: 'Oliver Watts'}), (c:City {name: 'Reutlingen'})
CREATE (student)-[:LIVES_IN]->(c)
"""

# Relationship between student and university
student_university_rel = """
MATCH (student1:Student {name: 'John Doe'}), (student2:Student {name: 'Leon Marshall'}), (student3:Student {name: 'Oliver Watts'}), (u:University {name: 'DHBW Stuttgart'})
CREATE (student1)-[:STUDIES_AT]->(u), (student2)-[:STUDIES_AT]->(u), (student3)-[:STUDIES_AT]->(u)
"""

# Create relationship between university and city node
graph.run(university_city_rel)
graph.run(professor_students_rel)
graph.run(professor_city_rel)
graph.run(professor_university_rel)
graph.run(student_city_rel_1)
graph.run(student_city_rel_2)
graph.run(student_city_rel_3)
graph.run(student_university_rel)

print("Relationships created successfully")

Relationships created successfully


### Update node properties

In [24]:
# Define query to get update person node properties
update_person_node = "MATCH (p:Person {name: 'John Doe', age: 25}) SET p.age = 26 RETURN DISTINCT p"

# Update person node properties in the database
graph.run(update_person_node)

# Define query to get updated person node
get_updated_person = "MATCH (p:Person {name: 'John Doe'}) RETURN p.name AS name, p.age AS age"

# Get updated person node from the database
result = graph.run(get_updated_person).to_data_frame()

# Print updated person node
result

,name,age
0,John Doe,20
